<a href="https://colab.research.google.com/github/Ch256981/IMLP347/blob/main/HW1_Ko_I_Hung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BUDT 758J - HW1

I-Hung Ko

In [2]:
# install PySpark
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
# create a SparkSession from pyspark.sql, which is the entry point to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [4]:
# verify
spark

Load Dataset - Customer

In [6]:
# Spark SQL
spark.read.json('hw1_customer.json').registerTempTable('customer')
spark.sql('SELECT c.* FROM customer c').show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+-------------+----------+--------------------+-------------+--------------------+-----------+
| customerCity|customerId|        customerName|customerState|      customerStreet|customerZip|
+-------------+----------+--------------------+-------------+--------------------+-----------+
|  Gainesville|         1|Contemporary Casuals|           FL|   1355 S Hines Blvd| 32601-2871|
|        Plano|         2|     Value Furniture|           TX| 15145 S.W. 17th St.| 75094-7743|
|       Albany|         3|    Home Furnishings|           NY|    1900 Allard Ave.| 12209-1125|
|     Carteret|         4|   Eastern Furniture|           NJ|   1925 Beltline Rd.| 07008-3188|
|   Sacramento|         5|         Impressions|           CA|   5585 Westcott Ct.| 94206-4056|
|      Boulder|         6|   Furniture Gallery|           CO|    325 Flatiron Dr.| 80514-4432|
|      Seattle|         7|    Period Furniture|           WA|     394 Rainbow Dr.| 97954-5589|
|  Santa Clara|         8| California Classics|   

In [7]:
# pandas dataframe
import pandas as pd
pd_df = pd.read_json('hw1_customer.json')
pd_df

,customerId,customerName,customerStreet,customerCity,customerState,customerZip
0,1,Contemporary Casuals,1355 S Hines Blvd,Gainesville,FL,32601-2871
1,2,Value Furniture,15145 S.W. 17th St.,Plano,TX,75094-7743
2,3,Home Furnishings,1900 Allard Ave.,Albany,NY,12209-1125
3,4,Eastern Furniture,1925 Beltline Rd.,Carteret,NJ,07008-3188
4,5,Impressions,5585 Westcott Ct.,Sacramento,CA,94206-4056
5,6,Furniture Gallery,325 Flatiron Dr.,Boulder,CO,80514-4432
6,7,Period Furniture,394 Rainbow Dr.,Seattle,WA,97954-5589
7,8,California Classics,816 Peach Rd.,Santa Clara,CA,96915-7754
8,9,M and H Casual Furniture,3709 First Street,Clearwater,FL,34620-2314
9,10,Seminole Interiors,2400 Rocky Point Dr.,Seminole,FL,34646-4423


In [8]:
# pyspark dataframe
sp_df = spark.read.json('hw1_customer.json')
sp_df.show()

+-------------+----------+--------------------+-------------+--------------------+-----------+
| customerCity|customerId|        customerName|customerState|      customerStreet|customerZip|
+-------------+----------+--------------------+-------------+--------------------+-----------+
|  Gainesville|         1|Contemporary Casuals|           FL|   1355 S Hines Blvd| 32601-2871|
|        Plano|         2|     Value Furniture|           TX| 15145 S.W. 17th St.| 75094-7743|
|       Albany|         3|    Home Furnishings|           NY|    1900 Allard Ave.| 12209-1125|
|     Carteret|         4|   Eastern Furniture|           NJ|   1925 Beltline Rd.| 07008-3188|
|   Sacramento|         5|         Impressions|           CA|   5585 Westcott Ct.| 94206-4056|
|      Boulder|         6|   Furniture Gallery|           CO|    325 Flatiron Dr.| 80514-4432|
|      Seattle|         7|    Period Furniture|           WA|     394 Rainbow Dr.| 97954-5589|
|  Santa Clara|         8| California Classics|   

Load Dataset - Order

In [44]:
# Spark SQL
spark.read.json('hw1_order.json').registerTempTable('order')
spark.sql('SELECT o.* FROM order o').show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+----------+----------+-------+
|customerId| orderDate|orderId|
+----------+----------+-------+
|         1|2010-10-21|   1001|
|         8|2010-10-21|   1002|
|        15|2010-10-22|   1003|
|         5|2010-10-22|   1004|
|         3|2010-10-24|   1005|
|         2|2010-10-24|   1006|
|        11|2010-10-27|   1007|
|        12|2010-10-30|   1008|
|         4|2010-11-05|   1009|
|         1|2010-11-05|   1010|
+----------+----------+-------+



In [45]:
# pandas dataframe
import pandas as pd
pd_df = pd.read_json('hw1_order.json')
pd_df

,orderId,customerId,orderDate
0,1001,1,2010-10-21
1,1002,8,2010-10-21
2,1003,15,2010-10-22
3,1004,5,2010-10-22
4,1005,3,2010-10-24
5,1006,2,2010-10-24
6,1007,11,2010-10-27
7,1008,12,2010-10-30
8,1009,4,2010-11-05
9,1010,1,2010-11-05


In [46]:
# pyspark dataframe
sp_df = spark.read.json('hw1_order.json')
sp_df.show()

+----------+----------+-------+
|customerId| orderDate|orderId|
+----------+----------+-------+
|         1|2010-10-21|   1001|
|         8|2010-10-21|   1002|
|        15|2010-10-22|   1003|
|         5|2010-10-22|   1004|
|         3|2010-10-24|   1005|
|         2|2010-10-24|   1006|
|        11|2010-10-27|   1007|
|        12|2010-10-30|   1008|
|         4|2010-11-05|   1009|
|         1|2010-11-05|   1010|
+----------+----------+-------+



EDA

In [9]:
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customerId      15 non-null     int64 
 1   customerName    15 non-null     object
 2   customerStreet  15 non-null     object
 3   customerCity    15 non-null     object
 4   customerState   15 non-null     object
 5   customerZip     15 non-null     object
dtypes: int64(1), object(5)
memory usage: 848.0+ bytes


In [10]:
pd_df.columns

Index(['customerId', 'customerName', 'customerStreet', 'customerCity',
       'customerState', 'customerZip'],
      dtype='object')

In [12]:
pd_df.describe()

,customerId
count,15.000000
mean,8.000000
std,4.472136
min,1.000000
25%,4.500000
50%,8.000000
75%,11.500000
max,15.000000


In [13]:
sp_df.columns

['customerCity',
 'customerId',
 'customerName',
 'customerState',
 'customerStreet',
 'customerZip']

In [14]:
sp_df.describe().show()

+-------+------------+----------------+--------------------+-------------+--------------+-----------+
|summary|customerCity|      customerId|        customerName|customerState|customerStreet|customerZip|
+-------+------------+----------------+--------------------+-------------+--------------+-----------+
|  count|          15|              15|                  15|           15|            15|         15|
|   mean|        NULL|             8.0|                NULL|         NULL|          NULL|       NULL|
| stddev|        NULL|4.47213595499958|                NULL|         NULL|          NULL|       NULL|
|    min|      Albany|               1|American Euro Lif...|           CA|112 Kiowai St.| 07008-3188|
|    max|    Seminole|              15|     Value Furniture|           WA| 816 Peach Rd.| 97954-5589|
+-------+------------+----------------+--------------------+-------------+--------------+-----------+



In [16]:
pd_df['customerId'].value_counts()

1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
Name: customerId, dtype: int64

In [18]:
pd_df.customerId.value_counts()

1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
Name: customerId, dtype: int64

Data Manipulation

1. How many customers in each of the state, New Jersey or New York?

In [19]:
# 703
spark.sql('''
SELECT c.customerState AS `Customer State`, COUNT(c.customerId) AS `Count of Customers`
FROM Customer c
GROUP BY c.customerState
HAVING c.customerState IN ('NJ', 'NY')
''').show()

+--------------+------------------+
|Customer State|Count of Customers|
+--------------+------------------+
|            NJ|                 2|
|            NY|                 1|
+--------------+------------------+



In [43]:
# 704
pd_df = pd_df[pd_df['customerState'].isin(['NJ', 'NY'])]
customer_count = pd_df.groupby('customerState')['customerId'].nunique().reset_index()
customer_count.columns = ['Customer State', 'Count of Customers']
customer_count

,Customer State,Count of Customers
0,NJ,2
1,NY,1


In [39]:
# 758J
from pyspark.sql.functions import col, countDistinct

filtered_df = sp_df.filter(col('customerState').isin(['NJ', 'NY']))

customer_count = filtered_df.groupBy('customerState')\
               .agg(countDistinct('customerId').alias('Count of Customers'))

customer_count = customer_count.withColumnRenamed('customerState', 'Customer State')

customer_count.show()

+--------------+------------------+
|Customer State|Count of Customers|
+--------------+------------------+
|            NJ|                 2|
|            NY|                 1|
+--------------+------------------+



2. How many orders in each month?

In [75]:
# 703
spark.sql('''
SELECT o.orderDate, COUNT (o.orderId) AS `Count of Orders`
FROM Order o
GROUP BY o.orderDate
ORDER BY o.orderDate ASC
''').show()

+----------+---------------+
| orderDate|Count of Orders|
+----------+---------------+
|2010-10-21|              2|
|2010-10-22|              2|
|2010-10-24|              2|
|2010-10-27|              1|
|2010-10-30|              1|
|2010-11-05|              2|
+----------+---------------+



In [71]:
# 704
orders_count = pd_df.groupby('orderDate')['orderId'].nunique().reset_index(name='Count of Orders')
orders_count

,orderDate,Count of Orders
0,2010-10-21,2
1,2010-10-22,2
2,2010-10-24,2
3,2010-10-27,1
4,2010-10-30,1
5,2010-11-05,2


In [87]:
# 758J
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct, to_date

spark = SparkSession.builder.appName("example").getOrCreate()

spark_df = spark.createDataFrame(pd_df)

orders_count = spark_df.withColumn('orderDate', to_date('orderDate')) \
                       .groupBy('orderDate') \
                       .agg(countDistinct('orderId').alias('Count of Orders'))

orders_count.show()


+----------+---------------+
| orderDate|Count of Orders|
+----------+---------------+
|2010-10-22|              2|
|2010-10-21|              2|
|2010-10-24|              2|
|2010-11-05|              2|
|2010-10-30|              1|
|2010-10-27|              1|
+----------+---------------+

